# License Plate Detection Accuracy Evaluation
Đánh giá độ chính xác phát hiện biển số xe của mô hình EasyOCR

In [ ]:
!pip install easyocr opencv-python shapely

In [5]:
import easyocr
import os
import cv2
import numpy as np
import warnings
from shapely.geometry import Polygon

# Tắt warnings không cần thiết
warnings.filterwarnings('ignore', category=UserWarning)

# Khởi tạo EasyOCR reader
reader = easyocr.Reader(['en'])
print("✅ EasyOCR reader initialized")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


✅ EasyOCR reader initialized


In [6]:
# Dataset path
dataset_path = "/workspaces/OCR/Vietnam-license-plate-1"
print(f"Dataset path: {dataset_path}")

# Kiểm tra số lượng ảnh test
test_images_path = os.path.join(dataset_path, 'test', 'images')
if os.path.exists(test_images_path):
    num_test_images = len([f for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
    print(f"Test set: {num_test_images} images")
else:
    print("❌ Test folder không tồn tại!")

Dataset path: /workspaces/OCR/Vietnam-license-plate-1
Test set: 101 images


In [7]:
def calculate_iou(box1, box2):
    """Calculate IoU between two bounding boxes"""
    try:
        poly1 = Polygon(box1)
        poly2 = Polygon(box2)

        if not poly1.is_valid or not poly2.is_valid:
            return 0.0

        intersection = poly1.intersection(poly2).area
        union = poly1.union(poly2).area

        return intersection / union if union > 0 else 0.0
    except:
        return 0.0

def parse_yolo_label(label_path, img_width, img_height):
    """Parse YOLO OBB label file"""
    objects = []

    if not os.path.exists(label_path):
        return objects

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 9:  # class + 8 coordinates
            # Convert normalized coordinates to pixel coordinates
            coords = []
            for i in range(1, 9, 2):  # x1,y1,x2,y2,x3,y3,x4,y4
                x = float(parts[i]) * img_width
                y = float(parts[i+1]) * img_height
                coords.append((x, y))
            objects.append({'bbox': coords})

    return objects

def evaluate_detection_accuracy(reader, dataset_path, iou_threshold=0.3):
    """Evaluate license plate detection accuracy"""
    image_folder = os.path.join(dataset_path, "test", "images")
    label_folder = os.path.join(dataset_path, "test", "labels")

    if not os.path.exists(image_folder) or not os.path.exists(label_folder):
        print("❌ Test folders không tồn tại!")
        return None

    total_images = 0
    correct_detections = 0

    image_files = [f for f in os.listdir(image_folder)
                   if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    print(f"🔍 Đánh giá {len(image_files)} ảnh...")

    for idx, filename in enumerate(image_files):
        img_path = os.path.join(image_folder, filename)
        label_path = os.path.join(label_folder, filename.rsplit('.', 1)[0] + '.txt')

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            continue

        total_images += 1
        img_height, img_width = image.shape[:2]

        # Parse ground truth
        gt_objects = parse_yolo_label(label_path, img_width, img_height)

        # Run OCR
        ocr_results = reader.readtext(image)

        # Check if any OCR detection overlaps with ground truth
        detected_correctly = False

        for bbox, text, conf in ocr_results:
            pred_bbox = [(float(point[0]), float(point[1])) for point in bbox]

            for gt_obj in gt_objects:
                iou = calculate_iou(pred_bbox, gt_obj['bbox'])
                if iou >= iou_threshold:
                    detected_correctly = True
                    break

            if detected_correctly:
                break

        if detected_correctly:
            correct_detections += 1

        # Progress indicator
        if (idx + 1) % 20 == 0:
            print(f"Đã xử lý {idx + 1}/{len(image_files)} ảnh...")

    # Calculate accuracy
    accuracy = correct_detections / total_images if total_images > 0 else 0

    return {
        'total_images': total_images,
        'correct_detections': correct_detections,
        'missed_detections': total_images - correct_detections,
        'accuracy': accuracy,
        'iou_threshold': iou_threshold
    }

# Run evaluation
print("📊 Bắt đầu đánh giá độ chính xác...")
results = evaluate_detection_accuracy(reader, dataset_path, iou_threshold=0.3)

if results:
    print("\n" + "="*50)
    print("🎯 KẾT QUẢ ĐÁNH GIÁ")
    print("="*50)
    print(f"📊 Tổng số ảnh test: {results['total_images']}")
    print(f"✅ Phát hiện đúng biển số: {results['correct_detections']}")
    print(f"❌ Không phát hiện được: {results['missed_detections']}")
    print(f"🎯 Độ chính xác: {results['accuracy']:.3f} ({results['accuracy']*100:.1f}%)")
    print(f"🔧 IoU threshold: {results['iou_threshold']}")
    print("="*50)
else:
    print("❌ Đánh giá thất bại!")

📊 Bắt đầu đánh giá độ chính xác...
🔍 Đánh giá 101 ảnh...
Đã xử lý 20/101 ảnh...
Đã xử lý 20/101 ảnh...
Đã xử lý 40/101 ảnh...
Đã xử lý 40/101 ảnh...
Đã xử lý 60/101 ảnh...
Đã xử lý 60/101 ảnh...
Đã xử lý 80/101 ảnh...
Đã xử lý 80/101 ảnh...
Đã xử lý 100/101 ảnh...
Đã xử lý 100/101 ảnh...

🎯 KẾT QUẢ ĐÁNH GIÁ
📊 Tổng số ảnh test: 101
✅ Phát hiện đúng biển số: 100
❌ Không phát hiện được: 1
🎯 Độ chính xác: 0.990 (99.0%)
🔧 IoU threshold: 0.3

🎯 KẾT QUẢ ĐÁNH GIÁ
📊 Tổng số ảnh test: 101
✅ Phát hiện đúng biển số: 100
❌ Không phát hiện được: 1
🎯 Độ chính xác: 0.990 (99.0%)
🔧 IoU threshold: 0.3
